# 📡 Proyecto Final: Telecom — Plan de Trabajo
**Jaime Chaves**

## 🎯 Objetivo del proyecto (recordatorio operativo)

Desarrollar un modelo de clasificación binaria que estime la probabilidad de churn de clientes de Interconnect, con el fin de identificar y priorizar clientes en riesgo para acciones preventivas de retención.

* Target: churn (derivado de EndDate)
* Salida principal: probabilidad de churn
* Métrica principal: AUC-ROC
* Métrica secundaria: Accuracy

## ETAPA 1 — Carga y comprensión de los datos

### 1.1 Carga de los datasets

Cargaremos los cuatro archivos, usando rutas robustas y manteniendo customerID como llave:

* contract = pd.read_csv(...)
* personal = pd.read_csv(...)
* internet = pd.read_csv(...)
* phone = pd.read_csv(...)


Objetivo:

* Confirmar tamaños, tipos de datos y presencia de nulos
* Verificar unicidad de customerID
* Entender qué información aporta cada fuente

### 1.2 Exploración inicial (EDA ligero)

Para cada dataframe:

* .info()
* .head()
* Conteo de valores nulos
* Distribución de variables categóricas clave

Para el target:

* Proporción churn / no churn
* Confirmar posible **desbalance de clases** (esperable)

**📌 Decisión clave temprana:**
El desbalance refuerza la elección de **AUC-ROC** como métrica principal.

## ETAPA 2 — Limpieza, transformación y construcción del dataset final

### 2.1 Tratamiento del target

* Crear variable binaria:
*   churn = 1 si EndDate != 'No'
*   churn = 0 si EndDate == 'No'
* Eliminar EndDate luego de crear el target (evitar leakage)

### 2.2 Limpieza por dataset y columnas útiles

**📄 contract.csv — Base del modelo**

Columnas útiles tras limpieza:
* Type → tipo de contrato (month-to-month / 1 year / 2 year)
* PaperlessBilling
* PaymentMethod
* MonthlyCharges
* TotalCharges (convertir a numérico)
* tenure (derivada de fechas)

Transformaciones clave:
* Convertir fechas
* Crear tenure_months
* Eliminar columnas puramente administrativas


**👤 personal.csv — Perfil del cliente**

Columnas útiles:
* gender
* SeniorCitizen
* Partner
* Dependents

Transformaciones:
* Convertir a categorías binarias
* No escalar (son categóricas)


**🌐 internet.csv — Servicios digitales**

Columnas útiles:
* InternetService (DSL / Fiber optic / No)
* OnlineSecurity
* OnlineBackup
* DeviceProtection
* TechSupport
* StreamingTV
* StreamingMovies

Transformaciones:
* Reemplazar “No internet service” → “No”
* Convertir a variables binarias

**📌 Insight esperado:** Clientes con muchos servicios activos tienden a menor churn (lock-in).


**☎️ phone.csv — Servicios de telefonía**

Columnas útiles:
* PhoneService
* MultipleLines

Transformaciones:
* Unificar valores categóricos
* Convertir a binario

### 2.3 Integración final

* Hacer merge progresivo por customerID
* Base final: 1 fila = 1 cliente
* Eliminar customerID antes del modelado (no predictiva)

## ETAPA 3 — Preparación para modelado

### 3.1 Encoding y escalado

* One-Hot Encoding para variables categóricas
* Escalado (StandardScaler) solo para:
* MonthlyCharges
* TotalCharges
* tenure_months

### 3.2 Split del dataset
* train_test_split
* test_size = 0.25
* random_state fijo
* Estratificación por churn

## ETAPA 4 — Modelado

### 4.1 Enfoque de modelado (alineado al objetivo)

El objetivo no es solo clasificar, sino ordenar clientes por riesgo de churn.

Por eso, los modelos deben:
* Entregar probabilidades confiables
* Funcionar bien con variables mixtas
* Manejar relaciones no lineales

### 4.2 Modelos a entrenar (progresión lógica)
🟦 Modelo base (baseline)
* **Logistic Regression**
* Ventaja:
* Interpretabilidad
* Punto de comparación obligatorio

🟩 Modelos principales **(esperados para alta AUC-ROC)**
* **Random Forest**
* **Gradient Boosting**
* **CatBoost** (muy recomendado)

📌 CatBoost es especialmente fuerte aquí porque:
* Maneja bien variables categóricas
* Es robusto con poco tuning
* Suele lograr AUC-ROC ≥ 0.85 en churn

### 4.3 Evaluación

Para cada modelo:
* AUC-ROC (principal)
* Accuracy (secundaria)
* Curva ROC
* Comparación clara en tabla

## ETAPA 5 — Interpretación y valor de negocio

### 5.1 Importancia de variables
* Feature importance
* Qué factores aumentan el riesgo de churn

Ejemplos esperados:
* Contratos mensuales
* Fibra óptica + alto costo
* Falta de servicios de soporte

### 5.2 Enfoque de negocio
* El modelo prioriza clientes, no decide promociones
* Permite:
* Segmentar campañas
* Reducir costo de retención
* Actuar antes del churn real

## ETAPA 6 — Informe final

El informe debe:
* Explicar el problema en lenguaje de negocio
* Justificar decisiones técnicas
* Presentar resultados con métricas
* Concluir con impacto práctico

### ✅ Resultado esperado
* Un modelo con AUC-ROC ≥ 0.85 (ideal ≥ 0.87)
* Notebook claro, reproducible y bien narrado
* Proyecto alineado con criterios de evaluación y realidad empresarial